In [1]:
# lr:0.001
# anneal_rate:0.9
# batch_size:32
# clip_norm:50
# num_epochs:5
# alpha:250
# beta:0
# max_beta:1
# step_beta:0.002
# anneal_iter:40000
# kl_anneal_iter:2000
# print_iter:100
# save_iter:5000
# num_workers:4

In [2]:
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install -q dive-into-graphs

In [3]:
!pip install -q toolz
!pip install -q wandb

In [4]:
%load_ext autoreload
%autoreload 2
import os
import json
import argparse
import pickle 

import numpy as np
import pandas as pd
import torch

from molecule_optimizer.externals.fast_jtnn.datautils import SemiMolTreeFolder, SemiMolTreeFolderTest
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from torch_geometric.data import DenseDataLoader

import rdkit

lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

import warnings
warnings.filterwarnings("ignore")

import math

In [5]:
conf = json.load(open("training/configs/rand_gen_zinc250k_config_dict.json"))

In [6]:
csv = pd.read_csv("ZINC_310k.csv")

In [7]:
smiles = csv['SMILES']

In [8]:
smiles = smiles[:60000]

In [9]:
labels = torch.tensor(csv['QED'][:60000]).float()

In [10]:
# if 'runner.xml' not in os.listdir("."):
#     runner = SemiJTVAEGeneratorPredictor(smiles)
#     with open('runner.xml', 'wb') as f:
#         pickle.dump(runner, f)

In [11]:
if 'runner_20.xml' not in os.listdir("."):
    runner = SemiJTVAEGeneratorPredictor(smiles)
    with open('runner_20.xml', 'wb') as f:
        pickle.dump(runner, f)

In [12]:
# with open('runner.xml', 'rb') as f:
#     runner = pickle.load(f)

In [13]:
with open('runner_20.xml', 'rb') as f:
    runner = pickle.load(f)

In [14]:
runner.get_model(
    "rand_gen",
    {
        "hidden_size": conf["model"]["hidden_size"],
        "latent_size": conf["model"]["latent_size"],
        "depthT": conf["model"]["depthT"],
        "depthG": conf["model"]["depthG"],
        "label_size": 1,
        "label_mean": float(torch.mean(labels)),
        "label_var": float(torch.var(labels)),
    },
)

In [15]:
labels = runner.get_processed_labels(labels)
preprocessed = runner.processed_smiles

In [16]:
# N_TEST = 10000
N_TEST = 200
VAL_FRAC = 0.05

In [17]:
perm_id=np.random.permutation(len(labels))
X_train = preprocessed[perm_id[N_TEST:]]
L_train = torch.tensor(labels.numpy()[perm_id[N_TEST:]])


X_test = preprocessed[perm_id[:N_TEST]]
L_test = torch.tensor(labels.numpy()[perm_id[:N_TEST]])

In [18]:
val_cut = math.floor(len(X_train) * VAL_FRAC)

In [19]:
X_Val = X_train[:val_cut]
L_Val = L_train[:val_cut]

X_train = X_train[val_cut :]
L_train = L_train[val_cut :]

In [20]:
train_loader = SemiMolTreeFolder(
    X_train,
    L_train,
    runner.vocab,
    conf["batch_size"],
    label_pct=0.05,
    num_workers=conf["num_workers"],
)

In [21]:
test_loader = SemiMolTreeFolderTest(
    X_test,
    L_test,
    runner.vocab,
    conf["batch_size"],
    num_workers=conf["num_workers"],
)

In [22]:
batch_size_val = math.floor(len(X_Val) / 10)

val_loader = SemiMolTreeFolderTest(
    X_Val,
    L_Val,
    runner.vocab,
    batch_size_val,
    num_workers=conf["num_workers"],
)

In [ ]:
print("Training model...")
runner.train_gen_pred(
    loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
)

Training model...
Model #Params: 4732K
[Train][100] Alpha: 66.503, Beta: 0.000, Loss: 177.01, KL: 87.75, MAE: 0.11276, Word Loss: 92.26, Topo Loss: 25.66, Assm Loss: 9.05, Pred Loss: 0.80, Word: 27.78, Topo: 79.16, Assm: 54.35, PNorm: 101.68, GNorm: 50.00
[Train][200] Alpha: 61.574, Beta: 0.000, Loss: 111.62, KL: 119.35, MAE: 0.09095, Word Loss: 58.46, Topo Loss: 14.27, Assm Loss: 8.59, Pred Loss: 0.54, Word: 47.80, Topo: 89.93, Assm: 58.65, PNorm: 105.76, GNorm: 50.00
[Train][300] Alpha: 77.440, Beta: 0.000, Loss: 99.73, KL: 147.30, MAE: 0.08573, Word Loss: 50.65, Topo Loss: 12.07, Assm Loss: 8.30, Pred Loss: 0.48, Word: 54.54, Topo: 91.30, Assm: 58.27, PNorm: 108.72, GNorm: 50.00
[Train][400] Alpha: 28.909, Beta: 0.000, Loss: 95.02, KL: 179.85, MAE: 0.08182, Word Loss: 47.61, Topo Loss: 11.54, Assm Loss: 8.16, Pred Loss: 0.43, Word: 57.59, Topo: 91.76, Assm: 60.09, PNorm: 111.56, GNorm: 50.00
[Train][500] Alpha: 28.293, Beta: 0.000, Loss: 86.89, KL: 202.06, MAE: 0.08202, Word Loss: 4

In [ ]:
print("Training model...")
runner.train_gen_pred_supervised(
    loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
)